In [1]:
from google.colab import drive
drive.mount('/content/drive')
filepath = '/content/drive/MyDrive/Baka/'

Mounted at /content/drive


In [2]:
##Importing stuff
import sympy as sp
import numpy as np

sp.init_printing()  # pretty printing in notebook

In [3]:
## Constants and variables

c = sp.symbols('c')  # speed of light
epsilon = sp.symbols(r'\varepsilon')
x,y,z,xp,yp,zp,omega,v, k = sp.symbols('x y z x_p y_p z_p omega v k')   # position, position of specimen, freq. of photon, speed of electron
gamma = 1/sp.sqrt( 1 - (v/c)**2 )
gamma = sp.symbols(r'\gamma_{L}')

iu = sp.I

We shall  start by defining Green's interaction operator $\mathrm{\mathbf{\hat{G}_{EE}^{int}}}$ of analytical form:
$$
\mathrm{\mathbf{\hat{G}_{EE}^{int}}} = \dfrac{2}{4 \pi \varepsilon_0} ( k^2 \mathrm{\mathbf{\hat{1}}} + ∇ _{\mathbf{r_p}} ⊗ ∇ _{\mathbf{r_p}} ) \mathrm{e}^{\mathrm{i} \ \omega \ {z_p}/{v} }
K_0 \left(  \dfrac{ \omega | \mathrm{\mathbb{r_{\perp}}} - \mathrm{\mathbb{r_{p,\perp}}} |^2 }{ v \gamma_\mathrm{L} }   \right),
$$
where

${\mathbf{\hat{1}}}$ ... unit tensor

$∇ _{\mathbf{r_p}} ⊗ ∇ _{\mathbf{r_p}}$ ... tensor product of two nablas in respect to the position of the pointlike $\mathrm{\mathbb{r_{p}}} = (\mathrm{\mathbb{r_{p,\perp}}},z_\mathrm{p} )$

 
$K_0$ ...  modified Bessel function of the second kind of order 0

$\gamma_L = 1/\sqrt{1-(v/c)^2}$ ... Lorentz factor



---


First we need sympy expression which can be derivated:
$$\mathrm{e}^{\mathrm{i} \ \omega \ {z_p}/{v} }
K_0 \left(  \dfrac{ \omega | \mathrm{\mathbb{r_{\perp}}} - \mathrm{\mathbb{r_{p,\perp}}} |^2 }{ v \gamma_\mathrm{L} }   \right),$$

In [4]:
GreenEE_preder = sp.exp(iu * omega * zp/v) * sp.besselk(0, omega * ( (x-xp)**2 + (y-yp)**2 ) / (v * gamma))
GreenEE_preder

In [5]:
# Generation of the full tensor

GreenEE_xx = sp.diff(GreenEE_preder, xp, xp)
GreenEE_xy = sp.diff(GreenEE_preder, xp, yp)
GreenEE_xz = sp.diff(GreenEE_preder, xp, zp)
GreenEE_yy = sp.diff(GreenEE_preder, yp, yp)
GreenEE_yz = sp.diff(GreenEE_preder, yp, zp)
GreenEE_zz = sp.diff(GreenEE_preder, zp, zp)

matice = sp.Matrix([[GreenEE_xx, GreenEE_xy, GreenEE_xz],
                    [GreenEE_xy, GreenEE_yy, GreenEE_yz],
                    [GreenEE_xz, GreenEE_yz, GreenEE_zz]])

eye = sp.eye(3)

matice = matice + k**2 * eye

matice = 1/(2 * sp.pi * epsilon) * matice

GreenEE = matice

'''
def GreenEE_setparticlepos(xpos, ypos, zpos):
    xx = GreenEE_xx.subs([(xp,xpos), (yp,ypos), (zp,zpos)])
    xy = GreenEE_xy.subs([(xp,xpos), (yp,ypos), (zp,zpos)])
    xz = GreenEE_xz.subs([(xp,xpos), (yp,ypos), (zp,zpos)])
    yy = GreenEE_yy.subs([(xp,xpos), (yp,ypos), (zp,zpos)])
    yz = GreenEE_yz.subs([(xp,xpos), (yp,ypos), (zp,zpos)])
    zz = GreenEE_zz.subs([(xp,xpos), (yp,ypos), (zp,zpos)])

    matice = sp.Matrix([[xx, xy, xz],
                        [xy, yy, yz],
                        [xz, yz, zz]])
    
    eye = sp.eye(3)

    matice = matice + k**2 * eye

    matice = 1/(2 * sp.pi * epsilon) * matice
    
    return matice
'''

'\ndef GreenEE_setparticlepos(xpos, ypos, zpos):\n    xx = GreenEE_xx.subs([(xp,xpos), (yp,ypos), (zp,zpos)])\n    xy = GreenEE_xy.subs([(xp,xpos), (yp,ypos), (zp,zpos)])\n    xz = GreenEE_xz.subs([(xp,xpos), (yp,ypos), (zp,zpos)])\n    yy = GreenEE_yy.subs([(xp,xpos), (yp,ypos), (zp,zpos)])\n    yz = GreenEE_yz.subs([(xp,xpos), (yp,ypos), (zp,zpos)])\n    zz = GreenEE_zz.subs([(xp,xpos), (yp,ypos), (zp,zpos)])\n\n    matice = sp.Matrix([[xx, xy, xz],\n                        [xy, yy, yz],\n                        [xz, yz, zz]])\n    \n    eye = sp.eye(3)\n\n    matice = matice + k**2 * eye\n\n    matice = 1/(2 * sp.pi * epsilon) * matice\n    \n    return matice\n'

In [6]:
GreenEE

⎡         ⎛                                                    ⎛       ⎛     ⎛
⎢         ⎜                                                  2 ⎜       ⎜   ω⋅⎝
⎢         ⎜       ⎛     ⎛        2           2⎞⎞   ω⋅(x - xₚ) ⋅⎜besselk⎜0, ───
⎢         ⎜       ⎜   ω⋅⎝(x - xₚ)  + (y - yₚ) ⎠⎟               ⎝       ⎝      
⎢     2⋅ω⋅⎜besselk⎜1, ─────────────────────────⎟ - ───────────────────────────
⎢ 2       ⎝       ⎝          \gamma_{L}⋅v      ⎠                              
⎢k  - ────────────────────────────────────────────────────────────────────────
⎢                                                                        \gamm
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                   2⋅π⋅\varep
⎢                                                                             
⎢                                                                             
⎢                                                   

Now, we will continue with Green's interaction operator $\mathrm{\mathbf{\hat{G}_{EM}^{int}}}$ of analytical form:
$$
\mathrm{\mathbf{\hat{G}_{EM}^{int}}} = 
\dfrac{1}{\mathrm{i}\, k\, c} ∇ _{\mathbf{r_p}} \times \mathrm{\mathbf{\hat{G}_{EE}^{int}}}
$$
where

$$
(\nabla \times \hat{A})_{il} = \varepsilon_{ijk} ∂_j A_{kl}
$$

In [7]:
def LeviCivita(i,j,k):
    if (i,j,k) in ((1,2,0),(2,0,1),(0,1,2)):
        output = 1
    elif (i,j,k) in ((1,0,2),(0,2,1),(2,1,0)):
        output = -1
    else:
        output = 0
    return output

In [20]:
xyzp = (xp,yp,zp)
ijk = range(3)

matice = sp.zeros(3)
for i in ijk:
    for l in ijk:
        for j in ijk:
            for k in ijk:
                matice[i,l] += LeviCivita(i,j,k) * sp.diff(GreenEE[k,l], xyzp[j])
 

GreenEM = 1/(iu * k * c) * matice

In [24]:
GreenEM

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

$$
\mathcal{G}(\mathbf{r}, \mathbf{r'}, \omega) = 
\begin{pmatrix}
\mathbf{\hat{G}_{EE}^{int}(r_p-r')}     &    \mathbf{\hat{G}_{EM}^{int}}     \\
\mathbf{\hat{G}_{ME}^{int}}     &    \mathbf{\hat{G}_{MM}^{int}} 
\end{pmatrix}
\cdot
\mathrm{Im}\underbrace{
\begin{pmatrix}
\mathbf{\hat{α}_{EE}}     &    \mathbf{\hat{α}_{EM}}     \\
\mathbf{\hat{α}_{ME}}     &    \mathbf{\hat{α}_{MM}} 
\end{pmatrix}
}_{\mathbb{\alpha}}
\cdot
\begin{pmatrix}
\mathbf{\hat{G}_{EE}^{int} (r-r_p) }     &    \mathbf{\hat{G}_{EM}^{int}}     \\
\mathbf{\hat{G}_{ME}^{int}}     &    \mathbf{\hat{G}_{MM}^{int}} 
\end{pmatrix}
$$

$$
\mathcal{G}_\mathrm{EE}(\mathbf{r}, \mathbf{r'}, \omega) = \mathbf{\hat{G}_{Ei}^{int}(r_p-r')} \cdot \mathrm{Im} [ \mathbf{\hat{α}_{ij}} ] \cdot \mathbf{\hat{G}_{jE}^{int}(r_p-r')} 
$$
$$
\mathbf{\hat{α}_{EM}} = - \mathbf{\hat{α}_{ME}}^T
$$

Let's now define the alpha matrix

**This takes a lot of time**

In [10]:
alpha_EE = sp.MatrixSymbol(r"\alpha_{EE}", 3, 3)
alpha_MM = sp.MatrixSymbol(r"\alpha_{MM}", 3, 3)
alpha_EM = sp.MatrixSymbol(r"\alpha_{EM}", 3, 3)
alpha_ME = - alpha_EM.T

superAlpha =   sp.BlockMatrix([[ alpha_EE, alpha_EM ],
                [ alpha_ME, alpha_MM ]])

superGreen = sp.BlockMatrix([[GreenEE, GreenEM],
              [-GreenEM, GreenEE]])

finalGreen = superGreen * superAlpha * superGreen

finalGreenEE = finalGreen[0:3,0:3]

In [11]:
## save the Green tensor to file for further use
import pickle

GreenFile = open(filepath+'GreenEE.dat', 'wb')
pickle.dump(finalGreenEE, GreenFile)
GreenFile.close()

In [12]:
finalGreenEE.atoms(sp.Symbol, sp.MatrixSymbol)

In [14]:
finalGreenEE.simplify()

⎛⎡⎡         ⎛                                                    ⎛       ⎛    
⎜⎢⎢         ⎜                                                  2 ⎜       ⎜   ω
⎜⎢⎢         ⎜       ⎛     ⎛        2           2⎞⎞   ω⋅(x - xₚ) ⋅⎜besselk⎜0, ─
⎜⎢⎢         ⎜       ⎜   ω⋅⎝(x - xₚ)  + (y - yₚ) ⎠⎟               ⎝       ⎝    
⎜⎢⎢     2⋅ω⋅⎜besselk⎜1, ─────────────────────────⎟ - ─────────────────────────
⎜⎢⎢ 2       ⎝       ⎝          \gamma_{L}⋅v      ⎠                            
⎜⎢⎢k  - ──────────────────────────────────────────────────────────────────────
⎜⎢⎢                                                                        \ga
⎜⎢⎢───────────────────────────────────────────────────────────────────────────
⎜⎢⎢                                                                   2⋅π⋅\var
⎜⎢⎢                                                                           
⎜⎢⎢                                                                           
⎜⎢⎢                                                 

In [ ]:
myEM = sp.Matrix([[0,0,0],
                  [0,0,0],
                  [0,0,1]])

finalGreenEE.subs([( alpha_EE, sp.zeros(3,3) ),
                   ( alpha_MM, sp.zeros(3,3) ),
                   ( alpha_EM, myEM )])
finalGreenEE

                                                                              
                                                                              
                                                                              
                                                                              
                                                                              
  ⎛                                     ⎛         ⎛           ________        
  ⎜                                     ⎜         ⎜          ╱      2         
  ⎜                                     ⎜         ⎜         ╱      v   ⎛      
  ⎜                                     ⎜         ⎜   ω⋅   ╱   1 - ── ⋅⎝(x - x
  ⎜                                     ⎜         ⎜       ╱         2         
  ⎜                                     ⎜         ⎜     ╲╱         c          
  ⎜     ⎛     2⎞                        ⎜  besselk⎜0, ────────────────────────
  ⎜   3 ⎜    v ⎟                        ⎜         ⎝ 

In [ ]:
type(finalGreenEE)

sympy.matrices.expressions.slice.MatrixSlice